In [1]:
import pandas as pd
from tensorflow.keras.preprocessing.text import text_to_word_sequence
import re

In [2]:
gf = pd.read_csv("post_text6.csv")
gf

,Post,Title,Replies,Views,Comment#,Query,Contents,Len,Days,Link,visit,read,topic,posts,given,received,topic_c,post_c
0,1,About the Copter Logs -- deprecated category,1,1444,1,"Do not post here, please post in the *",You should probably change this,31,945,0,2,20m,9,23,0,0,1,2
1,2,Attention: Please download PIXHAWK logs from t...,0,2056,0,"In order to help users better, I need readable...",NaN,0,0,0,0,0,0,0,0,0,0,0
2,3,"Extracting pitch, roll & yaw data for image mo...",3,1136,1,"Aloha, I’m a grad student at the university of...",I’m interested in this as well. I’ll be doing...,278,1,0,0,< 1m,0,7,0,0,2,5
3,3,"Extracting pitch, roll & yaw data for image mo...",3,1136,2,"Aloha, I’m a grad student at the university of...",Did you figure out how/what logs to use to get...,79,172,0,131,10h,319,2.2k,0,0,6,11
4,3,"Extracting pitch, roll & yaw data for image mo...",3,1136,3,"Aloha, I’m a grad student at the university of...",I am having the same problems at the moment Ha...,183,2386,1,46,4h,71,553,0,0,2,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3466,888,Setting Throttle Mid,2,252,1,As mentioned in this link: * at what altitude ...,Not sure why you think altitude matters. Anyth...,105,0,0,1.3k,4d,1.4k,8.4k,19,144,106,1.3k
3467,888,Setting Throttle Mid,2,252,2,As mentioned in this link: * at what altitude ...,As they didn’t mentioned any altitude thats wh...,100,0,0,0,< 1m,0,16,0,0,6,10
3468,889,"3DR Y6 crash, failsafe RTL flip, logfile analy...",1,287,1,Hoping to get some expertise on deciphering my...,I should add that the copter is physically and...,704,2,0,32,3h,69,553,0,0,5,12
3469,890,Dataflash logging and performance,1,222,1,"Hello, I just read the documentation on datafl...","Hi bds, The easy way is simply to observe perf...",598,0,0,0,< 1m,0,101,0,0,4,97


In [3]:
gf = gf.dropna(axis=0)
numlist = list(range(len(gf)))
gf = gf.set_index(pd.Index(numlist))
gf

,Post,Title,Replies,Views,Comment#,Query,Contents,Len,Days,Link,visit,read,topic,posts,given,received,topic_c,post_c
0,1,About the Copter Logs -- deprecated category,1,1444,1,"Do not post here, please post in the *",You should probably change this,31,945,0,2,20m,9,23,0,0,1,2
1,3,"Extracting pitch, roll & yaw data for image mo...",3,1136,1,"Aloha, I’m a grad student at the university of...",I’m interested in this as well. I’ll be doing...,278,1,0,0,< 1m,0,7,0,0,2,5
2,3,"Extracting pitch, roll & yaw data for image mo...",3,1136,2,"Aloha, I’m a grad student at the university of...",Did you figure out how/what logs to use to get...,79,172,0,131,10h,319,2.2k,0,0,6,11
3,3,"Extracting pitch, roll & yaw data for image mo...",3,1136,3,"Aloha, I’m a grad student at the university of...",I am having the same problems at the moment Ha...,183,2386,1,46,4h,71,553,0,0,2,6
4,4,Would like to know about PID tuning of drones ...,1,812,1,I would like to know more about PID. The math ...,You might be able to find PID tuning videos on...,310,2233,1,2,13m,9,36,0,1,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3356,888,Setting Throttle Mid,2,252,1,As mentioned in this link: * at what altitude ...,Not sure why you think altitude matters. Anyth...,105,0,0,1.3k,4d,1.4k,8.4k,19,144,106,1.3k
3357,888,Setting Throttle Mid,2,252,2,As mentioned in this link: * at what altitude ...,As they didn’t mentioned any altitude thats wh...,100,0,0,0,< 1m,0,16,0,0,6,10
3358,889,"3DR Y6 crash, failsafe RTL flip, logfile analy...",1,287,1,Hoping to get some expertise on deciphering my...,I should add that the copter is physically and...,704,2,0,32,3h,69,553,0,0,5,12
3359,890,Dataflash logging and performance,1,222,1,"Hello, I just read the documentation on datafl...","Hi bds, The easy way is simply to observe perf...",598,0,0,0,< 1m,0,101,0,0,4,97


In [4]:
# if clean label -> Match = '1', otherwise, Match = '0'
gf.loc[:1096, "Match"] = '1'
gf[:1098]

,Post,Title,Replies,Views,Comment#,Query,Contents,Len,Days,Link,visit,read,topic,posts,given,received,topic_c,post_c,Match
0,1,About the Copter Logs -- deprecated category,1,1444,1,"Do not post here, please post in the *",You should probably change this,31,945,0,2,20m,9,23,0,0,1,2,1
1,3,"Extracting pitch, roll & yaw data for image mo...",3,1136,1,"Aloha, I’m a grad student at the university of...",I’m interested in this as well. I’ll be doing...,278,1,0,0,< 1m,0,7,0,0,2,5,1
2,3,"Extracting pitch, roll & yaw data for image mo...",3,1136,2,"Aloha, I’m a grad student at the university of...",Did you figure out how/what logs to use to get...,79,172,0,131,10h,319,2.2k,0,0,6,11,1
3,3,"Extracting pitch, roll & yaw data for image mo...",3,1136,3,"Aloha, I’m a grad student at the university of...",I am having the same problems at the moment Ha...,183,2386,1,46,4h,71,553,0,0,2,6,1
4,4,Would like to know about PID tuning of drones ...,1,812,1,I would like to know more about PID. The math ...,You might be able to find PID tuning videos on...,310,2233,1,2,13m,9,36,0,1,1,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1093,299,AltHold not responding,2,459,2,Hello! My hexa is no longer holding altitude i...,* Sent using Tapatalk,21,19,1,3,7m,4,123,0,0,16,92,1
1094,300,Crash log analyis help!,3,570,1,Had a nice crash earlier today. Looking at the...,FPV Video of crash. [youtube]*,30,1,1,0,< 1m,0,11,0,0,4,7,1
1095,300,Crash log analyis help!,3,570,2,Had a nice crash earlier today. Looking at the...,Attached TLOG of flight too,27,1,0,0,< 1m,0,11,0,0,4,7,1
1096,300,Crash log analyis help!,3,570,3,Had a nice crash earlier today. Looking at the...,I think i found the issue. the back right mott...,74,1,0,0,< 1m,0,11,0,0,4,7,1


In [5]:
#  Noisy label -> '0'
I=[10,11,14,22,27,29,34,36,37,38,39,40,41,42,43,44,45,46,48,50,53,62,63,64,66,68,69,70,72,74,76,78,82,83,86,92,94,102,
   103,107,112,114,116,117,125,144,145,146,148,149,150,158,163,167,171,173,180,182,185,186,194,196,197,209,210,212,213,
   214,219,224,232,238,242,246,247,250,252,255,261,264,267,277,278,281,283,286,287,293,295,303,307,312,314,316,318,319,
   321,322,323,324,329,332,336,341,343,349,352,354,357,358,360,361,362,365,366,367,369,374,380,385,388,390,392,397,400,
   403,404,406,407,412,416,417,420,423,432,435,437,439,441,447,448,453,459,460,464,467,473,476,477,478,483,485,488,491,
   492,494,495,504,505,507,510,511,512,514,516,521,525,526,529,532,534,536,539,542,543,546,547,549,551,552,553,561,569,
   570,573,577,578,581,583,586,589,594,595,596,598,602,603,604,605,606,607,608,614,619,621,622,624,627,630,635,641,644,
   647,654,655,657,658,659,660,661,671,674,677,681,683,689,690,691,692,699,700,704,705,709,711,723,724,727,730,736,737,
   739,740,744,746,749,761,763,766,767,768,770,776,777,780,783,784,786,788,789,793,797,799,800,802,805,807,809,810,813,
   819,821,826,828,829,835,837,839,843,846,854,855,862,863,870,873,876,878,879,881,882,883,884,886,891,893,895,897,898,
   899,900,902,903,911,912,916,920,928,931,936,938,940,942,959,963,966,970,971,974,976,979,982,984,987,988,990,994,997,
   1005,1006,1013,1024,1026,1027,1029,1031,1035,1036,1038,1043,1045,1050,1053,1055,1061,1065,1069,1070,1073,1077,1079,
   1080,1081,1082,1086,1088,1089,1094,1095,1096]

L = []
for i in range(1, 1098):
    if i in I: L.append('0')
    else: L.append('1')

for j in range(1097):
    gf.iloc[j,-1] = L[j]
gf[:1097]

,Post,Title,Replies,Views,Comment#,Query,Contents,Len,Days,Link,visit,read,topic,posts,given,received,topic_c,post_c,Match
0,1,About the Copter Logs -- deprecated category,1,1444,1,"Do not post here, please post in the *",You should probably change this,31,945,0,2,20m,9,23,0,0,1,2,1
1,3,"Extracting pitch, roll & yaw data for image mo...",3,1136,1,"Aloha, I’m a grad student at the university of...",I’m interested in this as well. I’ll be doing...,278,1,0,0,< 1m,0,7,0,0,2,5,1
2,3,"Extracting pitch, roll & yaw data for image mo...",3,1136,2,"Aloha, I’m a grad student at the university of...",Did you figure out how/what logs to use to get...,79,172,0,131,10h,319,2.2k,0,0,6,11,1
3,3,"Extracting pitch, roll & yaw data for image mo...",3,1136,3,"Aloha, I’m a grad student at the university of...",I am having the same problems at the moment Ha...,183,2386,1,46,4h,71,553,0,0,2,6,1
4,4,Would like to know about PID tuning of drones ...,1,812,1,I would like to know more about PID. The math ...,You might be able to find PID tuning videos on...,310,2233,1,2,13m,9,36,0,1,1,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1092,299,AltHold not responding,2,459,1,Hello! My hexa is no longer holding altitude i...,I probably found it! It’s a problem I didn’t s...,301,18,0,3,7m,4,123,0,0,16,92,1
1093,299,AltHold not responding,2,459,2,Hello! My hexa is no longer holding altitude i...,* Sent using Tapatalk,21,19,1,3,7m,4,123,0,0,16,92,0
1094,300,Crash log analyis help!,3,570,1,Had a nice crash earlier today. Looking at the...,FPV Video of crash. [youtube]*,30,1,1,0,< 1m,0,11,0,0,4,7,0
1095,300,Crash log analyis help!,3,570,2,Had a nice crash earlier today. Looking at the...,Attached TLOG of flight too,27,1,0,0,< 1m,0,11,0,0,4,7,0


In [6]:
# Labeling(helpful)
gf.loc[:1096, "Label"] = '0'

# Correct of Helpful(MODE(Doheon,Paul,John), 649)
I=[ # Train data
   5,6,8,9,13,14,15,16,20,21,23,25,27,30,32,34,36,38,40,42,44,46,48,49,50,51,52,53,54,55,
   57,58,59,60,62,63,66,68,69,70,76,78,80,81,83,84,85,86,90,91,92,93,94,95,96,99,102,104,106,107,
   108,109,110,111,112,113,116,117,119,120,122,125,129,130,131,132,134,135,138,140,144,147,149,150,151,153,154,164,165,166,
   168,169,170,173,175,177,179,184,186,188,191,193,194,195,196,198,200,202,204,205,206,207,208,209,211,212,213,216,219,220,
   223,225,226,228,231,234,236,237,238,239,240,241,242,243,245,248,249,250,251,252,253,254,256,257,258,260,261,262,265,266,
   272,273,275,276,277,278,280,282,283,284,285,286,287,288,289,290,291,292,293,294,296,298,300,302,308,310,313,317,319,325,
   327,332,334,339,340,341,342,344,347,350,351,352,355,356,358,359,363,364,365,366,367,368,370,371,372,373,374,375,376,377,
   379,380,381,382,384,387,390,391,392,394,396,397,399,400,401,403,405,406,408,410,411,413,414,415,416,418,419,420,422,423,
   424,426,427,428,429,430,431,433,434,435,437,438,440,442,443,444,445,447,449,450,451,452,453,454,455,456,460,462,463,468,
   469,471,473,475,476,479,481,482,483,484,486,487,489,490,491,492,494,495,498,499,501,502,506,508,510,511,516,517,519,520,
   523,525,526,527,528,530,531,533,534,535,536,537,540,541,544,545,546,547,549,552,554,556,558,559,561,562,563,564,565,566,
   567,569,570,572,574,576,578,580,581,584,585,587,588,589,590,591,593,594,595,596,597,598,599,602,603,604,605,606,607,608,
   609,611,612,613,615,617,618,620,621,623,628,633,634,636,638,640,642,649,651,655,660,661,662,664,665,666,667,668,669,672,
   674,676,679,680,682,684,686,687,688,691,692,693,694,695,696,698,699,700,701,702,704,705,706,707,709,710,711,712,714,715,
   718,719,720,722,724,725,726,727,728,729,730,731,732,733,734,735,736,737,738,739,741,742,745,746,747,748,749,750,752,756,
   757,758,761,762,764,766,767,769,770,771,772,773,775,776,777,778,780,782,785,786,787,788,790,791,792,794,795,797,799,800,
   801,802,803,804,805,806,808,809,811,812,814,815,816,817,818,819,820,822,824,825,827,828,830,831,833,834,835,836,837,838,
   839,840,841,842,843,846,849,850,851,852,853,855,859,860,862,865,866,871,872,874,877,
   
   # Test data 
   878,879,880,883,885,886,887,888,890,891,892,893,897,901,902,903,904,905,906,907,910,912,913,921,922,923,924,925,926,927,
   929,930,931,934,936,937,939,940,944,946,947,948,950,953,954,955,957,959,960,961,962,963,964,965,968,969,970,971,978,979,
   981,982,983,989,993,994,996,997,999,1002,1003,1004,1006,1008,1009,1015,1016,1018,1019,1020,1021,1024,1025,1030,1032,
   1036,1038,1039,1040,1041,1044,1046,1047,1050,1057,1059,1060,1062,1064,1066,1067,1071,1072,1074,1078,1081,1082,1083,
   1084,1085,1087,1088,1090,1091,1092,1093,1094,1097]


L = []
for i in range(1, 1098):
    if i in I: L.append('1')
    else: L.append('0')
#     print(i, L[i-1])
# print(L)

for j in range(1097):
    gf.iloc[j,-1] = L[j]
gf[:1097]

,Post,Title,Replies,Views,Comment#,Query,Contents,Len,Days,Link,visit,read,topic,posts,given,received,topic_c,post_c,Match,Label
0,1,About the Copter Logs -- deprecated category,1,1444,1,"Do not post here, please post in the *",You should probably change this,31,945,0,2,20m,9,23,0,0,1,2,1,0
1,3,"Extracting pitch, roll & yaw data for image mo...",3,1136,1,"Aloha, I’m a grad student at the university of...",I’m interested in this as well. I’ll be doing...,278,1,0,0,< 1m,0,7,0,0,2,5,1,0
2,3,"Extracting pitch, roll & yaw data for image mo...",3,1136,2,"Aloha, I’m a grad student at the university of...",Did you figure out how/what logs to use to get...,79,172,0,131,10h,319,2.2k,0,0,6,11,1,0
3,3,"Extracting pitch, roll & yaw data for image mo...",3,1136,3,"Aloha, I’m a grad student at the university of...",I am having the same problems at the moment Ha...,183,2386,1,46,4h,71,553,0,0,2,6,1,0
4,4,Would like to know about PID tuning of drones ...,1,812,1,I would like to know more about PID. The math ...,You might be able to find PID tuning videos on...,310,2233,1,2,13m,9,36,0,1,1,2,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1092,299,AltHold not responding,2,459,1,Hello! My hexa is no longer holding altitude i...,I probably found it! It’s a problem I didn’t s...,301,18,0,3,7m,4,123,0,0,16,92,1,1
1093,299,AltHold not responding,2,459,2,Hello! My hexa is no longer holding altitude i...,* Sent using Tapatalk,21,19,1,3,7m,4,123,0,0,16,92,0,1
1094,300,Crash log analyis help!,3,570,1,Had a nice crash earlier today. Looking at the...,FPV Video of crash. [youtube]*,30,1,1,0,< 1m,0,11,0,0,4,7,0,0
1095,300,Crash log analyis help!,3,570,2,Had a nice crash earlier today. Looking at the...,Attached TLOG of flight too,27,1,0,0,< 1m,0,11,0,0,4,7,0,0


In [7]:
# Clean '1', Noisy '0'
gf['Match'].value_counts()

1    737
0    360
Name: Match, dtype: int64

In [8]:
Z=[]
for i in range(1,301):
    z=[]
    Q=gf[gf['Post']==i][['Post','Match']]
    if list(Q['Post']).count(i) != 0:
#         print(i, list(Q['Match']).count('1')/list(Q['Post']).count(i) )
        z.append(i)
        z.append(list(Q['Match']).count('1')/list(Q['Post']).count(i))
        z.append(list(Q['Post']).count(i))
    else: 
#         print(i,0)
        z.append(i)
        z.append(0)
        z.append(list(Q['Post']).count(i))
    Z.append(z)
Z    

[[1, 1.0, 1],
 [2, 0, 0],
 [3, 1.0, 3],
 [4, 1.0, 1],
 [5, 0.8, 15],
 [6, 0.6666666666666666, 3],
 [7, 1.0, 3],
 [8, 0.3181818181818182, 22],
 [9, 0.75, 8],
 [10, 0.5833333333333334, 12],
 [11, 0.42857142857142855, 7],
 [12, 0.6428571428571429, 14],
 [13, 0.7, 30],
 [14, 0.8888888888888888, 9],
 [15, 1.0, 8],
 [16, 0.7777777777777778, 9],
 [17, 0.6363636363636364, 11],
 [18, 0.75, 4],
 [19, 0.7272727272727273, 11],
 [20, 1.0, 1],
 [21, 0.7272727272727273, 11],
 [22, 0.5, 4],
 [23, 1.0, 6],
 [24, 0.5, 2],
 [25, 0.6666666666666666, 24],
 [26, 1.0, 3],
 [27, 0.8571428571428571, 7],
 [28, 1.0, 1],
 [29, 0.5, 2],
 [30, 0.875, 8],
 [31, 0.75, 4],
 [32, 0.5, 2],
 [33, 0.5, 2],
 [34, 1.0, 1],
 [35, 0.0, 1],
 [36, 0, 0],
 [37, 0.75, 4],
 [38, 0.6666666666666666, 3],
 [39, 1.0, 3],
 [40, 0.0, 1],
 [41, 0.7619047619047619, 21],
 [42, 0, 0],
 [43, 0.6666666666666666, 9],
 [44, 0.5, 2],
 [45, 0.5, 2],
 [46, 1.0, 6],
 [47, 1.0, 1],
 [48, 0.0, 1],
 [49, 0, 0],
 [50, 0, 0],
 [51, 0.7777777777777778, 9

In [99]:
import pandas as pd
H=pd.DataFrame(Z, columns=['Post','Rate','Count'])
H

,Post,Rate,Count
0,1,1.000000,1
1,2,0.000000,0
2,3,1.000000,3
3,4,1.000000,1
4,5,0.800000,15
...,...,...,...
295,296,0.000000,0
296,297,0.000000,0
297,298,0.666667,6
298,299,0.500000,2


In [10]:
C=pd.DataFrame(Z, columns=['Thread','C_Level','Comment'])
C

,Thread,C_Level,Comment
0,1,1.000000,1
1,2,0.000000,0
2,3,1.000000,3
3,4,1.000000,1
4,5,0.800000,15
...,...,...,...
295,296,0.000000,0
296,297,0.000000,0
297,298,0.666667,6
298,299,0.500000,2


In [11]:
C=C.sort_values(by='C_Level', ascending=False)
C

,Thread,C_Level,Comment
0,1,1.0,1
136,137,1.0,5
133,134,1.0,3
250,251,1.0,3
129,130,1.0,2
...,...,...,...
154,155,0.0,1
156,157,0.0,2
236,237,0.0,0
235,236,0.0,1


# Split 3 and 4

In [12]:
H=H.sort_values(by='Rate', ascending=False) # Split 3
H

# H=H.sort_values(by='Rate', ascending=True)   # Split 4
# H

,Post,Rate,Count
0,1,1.0,1
136,137,1.0,5
133,134,1.0,3
250,251,1.0,3
129,130,1.0,2
...,...,...,...
154,155,0.0,1
156,157,0.0,2
236,237,0.0,0
235,236,0.0,1


In [129]:
for i in range(1,301):
    U=sum(list(H['Count'][:i]))
    print("percentage of train data: ", U/1097)
    # train
    R=H[:i]
    # test
    S=H[i:]
    RL=list(R['Post'].sort_values())
    print(len(RL))
    SL=list(S['Post'].sort_values())
    print(len(SL))
    init1=0
    for r in RL:
        W=gf[gf['Post']==r][['Label']]
        init1 += list(W['Label']).count('1')
    print(init1)
    init2=0
    for r in RL:
        W=gf[gf['Post']==r][['Label']]
        init2 += list(W['Label']).count('0')
    print(init2)
    init3=0
    for s in SL:
        W=gf[gf['Post']==s][['Label']]
        init3 += list(W['Label']).count('1')
    print(init3)
    init4=0
    for s in SL:
        W=gf[gf['Post']==s][['Label']]
        init4 += list(W['Label']).count('0')
    print(init4)
    if (init1+init2) == 0:
        print(0)
    else:    print(init1/(init1+init2)) # rate of '1' in train data
    if (init3+init4) == 0:
        print(0)
    else:    print(init3/(init3+init4)) # rate of '1' in test data
    print('*'*100)

percentage of train data:  0.0009115770282588879
1
299
0
1
649
447
0.0
0.5921532846715328
****************************************************************************************************
percentage of train data:  0.0036463081130355514
2
298
2
2
647
446
0.5
0.5919487648673376
****************************************************************************************************
percentage of train data:  0.0054694621695533276
3
297
3
3


KeyboardInterrupt: 

In [13]:
v=154  # Split 3 

# v=201 # Split 4

In [14]:
U=sum(list(H['Count'][:v]))
U

782

In [15]:
U/1097  # percentage of train data

0.7128532360984503

In [16]:
# train
R=H[:v]
# test
S=H[v:]

In [17]:
# train thread#
RL=list(R['Post'].sort_values())
print(len(RL),RL)

154 [1, 3, 4, 5, 6, 7, 9, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 23, 25, 26, 27, 28, 30, 31, 34, 37, 38, 39, 41, 43, 46, 47, 51, 57, 58, 60, 63, 64, 66, 68, 69, 73, 74, 75, 76, 77, 79, 81, 84, 85, 88, 89, 90, 94, 96, 98, 100, 104, 105, 106, 108, 111, 113, 114, 115, 116, 117, 119, 120, 122, 125, 126, 127, 129, 130, 132, 134, 136, 137, 138, 140, 143, 146, 147, 149, 152, 153, 156, 161, 163, 169, 175, 177, 178, 179, 180, 182, 186, 189, 190, 191, 194, 196, 197, 198, 204, 210, 212, 214, 217, 218, 219, 221, 222, 225, 226, 227, 228, 230, 231, 232, 234, 238, 239, 244, 245, 247, 248, 249, 250, 251, 254, 255, 257, 258, 261, 262, 263, 264, 265, 266, 269, 273, 274, 275, 276, 277, 279, 282, 283, 287, 289, 290, 298]


In [18]:
# test thread#
SL=list(S['Post'].sort_values())
print(len(SL),SL)

146 [2, 8, 10, 11, 22, 24, 29, 32, 33, 35, 36, 40, 42, 44, 45, 48, 49, 50, 52, 53, 54, 55, 56, 59, 61, 62, 65, 67, 70, 71, 72, 78, 80, 82, 83, 86, 87, 91, 92, 93, 95, 97, 99, 101, 102, 103, 107, 109, 110, 112, 118, 121, 123, 124, 128, 131, 133, 135, 139, 141, 142, 144, 145, 148, 150, 151, 154, 155, 157, 158, 159, 160, 162, 164, 165, 166, 167, 168, 170, 171, 172, 173, 174, 176, 181, 183, 184, 185, 187, 188, 192, 193, 195, 199, 200, 201, 202, 203, 205, 206, 207, 208, 209, 211, 213, 215, 216, 220, 223, 224, 229, 233, 235, 236, 237, 240, 241, 242, 243, 246, 252, 253, 256, 259, 260, 267, 268, 270, 271, 272, 278, 280, 281, 284, 285, 286, 288, 291, 292, 293, 294, 295, 296, 297, 299, 300]


In [21]:
t=pd.DataFrame(H["Rate"].value_counts()).sort_index(ascending=False)  # clean ration of all threads
t

,Rate
1.000000,78
0.888889,2
0.875000,1
0.857143,3
0.833333,1
0.800000,5
0.777778,3
0.769231,2
0.761905,1
0.750000,17


In [22]:
y=pd.DataFrame(R["Rate"].value_counts()).sort_index(ascending=False)  # clean ration of training threads
y

,Rate
1.000000,78
0.888889,2
0.875000,1
0.857143,3
0.833333,1
0.800000,5
0.777778,3
0.769231,2
0.761905,1
0.750000,17


In [23]:
u=pd.DataFrame(S["Rate"].value_counts()).sort_index(ascending=False)  # clean ration of test threads
u

,Rate
0.600000,2
0.583333,1
0.571429,3
0.555556,1
0.500000,38
0.444444,1
0.428571,3
0.400000,1
0.375000,1
0.363636,1


In [24]:
pd.concat([t,y,u], axis=1)  # clean ratios // Total, training, test 

,Rate,Rate,Rate
1.000000,78,78.0,NaN
0.888889,2,2.0,NaN
0.875000,1,1.0,NaN
0.857143,3,3.0,NaN
0.833333,1,1.0,NaN
0.800000,5,5.0,NaN
0.777778,3,3.0,NaN
0.769231,2,2.0,NaN
0.761905,1,1.0,NaN
0.750000,17,17.0,NaN


In [25]:
# label '1' in train
init1=0
for r in RL:
    W=gf[gf['Post']==r][['Label']]
    init1 += list(W['Label']).count('1')
init1

464

In [26]:
# label '0' in train
init2=0
for r in RL:
    W=gf[gf['Post']==r][['Label']]
    init2 += list(W['Label']).count('0')
init2

318

In [27]:
# label '1' in test
init3=0
for s in SL:
    W=gf[gf['Post']==s][['Label']]
    init3 += list(W['Label']).count('1')
init3

185

In [28]:
# label '0' in test
init4=0
for s in SL:
    W=gf[gf['Post']==s][['Label']]
    init4 += list(W['Label']).count('0')
init4

130

In [29]:
init1/(init1+init2) # rate of '1' in train data

0.5933503836317136

In [30]:
init3/(init3+init4) # rate of '1' in test data

0.5873015873015873

In [31]:
# train thread#
RL=list(R['Post'].sort_values())
print(len(RL),RL)

154 [1, 3, 4, 5, 6, 7, 9, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 23, 25, 26, 27, 28, 30, 31, 34, 37, 38, 39, 41, 43, 46, 47, 51, 57, 58, 60, 63, 64, 66, 68, 69, 73, 74, 75, 76, 77, 79, 81, 84, 85, 88, 89, 90, 94, 96, 98, 100, 104, 105, 106, 108, 111, 113, 114, 115, 116, 117, 119, 120, 122, 125, 126, 127, 129, 130, 132, 134, 136, 137, 138, 140, 143, 146, 147, 149, 152, 153, 156, 161, 163, 169, 175, 177, 178, 179, 180, 182, 186, 189, 190, 191, 194, 196, 197, 198, 204, 210, 212, 214, 217, 218, 219, 221, 222, 225, 226, 227, 228, 230, 231, 232, 234, 238, 239, 244, 245, 247, 248, 249, 250, 251, 254, 255, 257, 258, 261, 262, 263, 264, 265, 266, 269, 273, 274, 275, 276, 277, 279, 282, 283, 287, 289, 290, 298]


In [32]:
# test thread#
SL=list(S['Post'].sort_values())
print(len(SL),SL)

146 [2, 8, 10, 11, 22, 24, 29, 32, 33, 35, 36, 40, 42, 44, 45, 48, 49, 50, 52, 53, 54, 55, 56, 59, 61, 62, 65, 67, 70, 71, 72, 78, 80, 82, 83, 86, 87, 91, 92, 93, 95, 97, 99, 101, 102, 103, 107, 109, 110, 112, 118, 121, 123, 124, 128, 131, 133, 135, 139, 141, 142, 144, 145, 148, 150, 151, 154, 155, 157, 158, 159, 160, 162, 164, 165, 166, 167, 168, 170, 171, 172, 173, 174, 176, 181, 183, 184, 185, 187, 188, 192, 193, 195, 199, 200, 201, 202, 203, 205, 206, 207, 208, 209, 211, 213, 215, 216, 220, 223, 224, 229, 233, 235, 236, 237, 240, 241, 242, 243, 246, 252, 253, 256, 259, 260, 267, 268, 270, 271, 272, 278, 280, 281, 284, 285, 286, 288, 291, 292, 293, 294, 295, 296, 297, 299, 300]


# Split 5

In [80]:
# Original H, before sorting
H

,Post,Rate,Count
0,1,1.000000,1
1,2,0.000000,0
2,3,1.000000,3
3,4,1.000000,1
4,5,0.800000,15
...,...,...,...
295,296,0.000000,0
296,297,0.000000,0
297,298,0.666667,6
298,299,0.500000,2


In [81]:
H=H.sort_values(by='Rate', ascending=False)
H

,Post,Rate,Count
0,1,1.0,1
136,137,1.0,5
133,134,1.0,3
250,251,1.0,3
129,130,1.0,2
...,...,...,...
154,155,0.0,1
156,157,0.0,2
236,237,0.0,0
235,236,0.0,1


In [82]:
for v1 in range(105,118):
    for v2 in range(143,150):
        print("v1:", v1, "v2:", v2)
        U1=sum(list(H['Count'][:v1])) # train
        U2=sum(list(H['Count'][v1:v2])) # test
        U3=sum(list(H['Count'][v2:])) # train
        
        # train
        R=pd.concat([H[:v1], H[v2:]])
        # test
        S=H[v1:v2]
        RL=list(R['Post'].sort_values())
        
        SL=list(S['Post'].sort_values())
        
        init1=0
        for r in RL:
            W=gf[gf['Post']==r][['Label']]
            init1 += list(W['Label']).count('1')
        
        init2=0
        for r in RL:
            W=gf[gf['Post']==r][['Label']]
            init2 += list(W['Label']).count('0')
        
        init3=0
        for s in SL:
            W=gf[gf['Post']==s][['Label']]
            init3 += list(W['Label']).count('1')
        
        init4=0
        for s in SL:
            W=gf[gf['Post']==s][['Label']]
            init4 += list(W['Label']).count('0')
        
        if (U1+U3)/1097 > 0.78 and (U1+U3)/1097 < 0.82 and (init1/(init1+init2)) > 0.57 and (init1/(init1+init2)) < 0.61 and (init3/(init3+init4)) > 0.57 and (init3/(init3+init4))< 0.61 and (U1/U3) > 0.9 and (U1/U3) < 1.1:
            print("percentage of train data: ", (U1+U3)/1097) # 0.7~0.8 
            print("U1:", U1, "U3:", U3, "U1/U3:", U1/U3)  # 1
            print("RL:", len(RL))
            print("SL:", len(SL))
            print(init1)
            print(init2)
            print(init3)
            print(init4)
            if (init1+init2) == 0:  print(0)
            else:    print(init1/(init1+init2)) # rate of '1' in train data
            if (init3+init4) == 0:  print(0)
            else:    print(init3/(init3+init4)) # rate of '1' in test data
            print('*'*100)
        

v1: 105 v2: 143
v1: 105 v2: 144
v1: 105 v2: 145
v1: 105 v2: 146
v1: 105 v2: 147
v1: 105 v2: 148
v1: 105 v2: 149
v1: 106 v2: 143
v1: 106 v2: 144
v1: 106 v2: 145
v1: 106 v2: 146
v1: 106 v2: 147
v1: 106 v2: 148
v1: 106 v2: 149
v1: 107 v2: 143
v1: 107 v2: 144
v1: 107 v2: 145
v1: 107 v2: 146
v1: 107 v2: 147
v1: 107 v2: 148
v1: 107 v2: 149
v1: 108 v2: 143
v1: 108 v2: 144
v1: 108 v2: 145
v1: 108 v2: 146
v1: 108 v2: 147
v1: 108 v2: 148
v1: 108 v2: 149
v1: 109 v2: 143
v1: 109 v2: 144
v1: 109 v2: 145
v1: 109 v2: 146
v1: 109 v2: 147
v1: 109 v2: 148
v1: 109 v2: 149
v1: 110 v2: 143
v1: 110 v2: 144
v1: 110 v2: 145
v1: 110 v2: 146
v1: 110 v2: 147
v1: 110 v2: 148
v1: 110 v2: 149
v1: 111 v2: 143
v1: 111 v2: 144
v1: 111 v2: 145
v1: 111 v2: 146
v1: 111 v2: 147
v1: 111 v2: 148
v1: 111 v2: 149
v1: 112 v2: 143
v1: 112 v2: 144
v1: 112 v2: 145
v1: 112 v2: 146
v1: 112 v2: 147
v1: 112 v2: 148
v1: 112 v2: 149
v1: 113 v2: 143
v1: 113 v2: 144
v1: 113 v2: 145
v1: 113 v2: 146
v1: 113 v2: 147
v1: 113 v2: 148
v1: 113 

In [83]:
# # version 6
# v1=112
# v2=148

# Split 5    116
v1=116
v2=143

In [84]:
# H['Count'][:v1] // H['Count'][V1:v2] // H['Count'][v2:]
# 앞쪽 쓰레드(매칭률 높은 것)와 뒷쪽 쓰레드(매칭률 낮은 것)을 균등하게 합쳐서 트레이닝 데이터 구성.. 총 총 댓글 수는 760~개(70%)정도
U1=sum(list(H['Count'][:v1]))
U2=sum(list(H['Count'][v1:v2]))
U3=sum(list(H['Count'][v2:]))
print(U1, U3, U1+U3, (U1+U3)/1097)

433 449 882 0.8040109389243391


In [85]:
U2

215

In [86]:
# train
R=pd.concat([H[:v1], H[v2:]])
R
# test
S=H[v1:v2]

In [87]:
R

,Post,Rate,Count
0,1,1.0,1
136,137,1.0,5
133,134,1.0,3
250,251,1.0,3
129,130,1.0,2
...,...,...,...
154,155,0.0,1
156,157,0.0,2
236,237,0.0,0
235,236,0.0,1


In [88]:
sum(R['Count'])/1097

0.8040109389243391

In [89]:
# train thread#
RL=list(R['Post'].sort_values())
print(len(RL),RL)

273 [1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 12, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 39, 40, 41, 42, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 99, 100, 101, 102, 103, 104, 106, 107, 108, 109, 110, 111, 112, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 141, 142, 143, 144, 145, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 176, 177, 178, 179, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238

In [90]:
# test thread#
SL=list(S['Post'].sort_values())
print(len(SL),SL)

27 [6, 13, 25, 38, 43, 57, 75, 76, 98, 105, 113, 114, 126, 140, 146, 147, 175, 180, 214, 239, 247, 265, 269, 274, 276, 290, 298]


In [91]:
t=pd.DataFrame(H["Rate"].value_counts()).sort_index(ascending=False)  # 모든 thread의 클린레이블 비율
t

,Rate
1.000000,78
0.888889,2
0.875000,1
0.857143,3
0.833333,1
0.800000,5
0.777778,3
0.769231,2
0.761905,1
0.750000,17


In [92]:
y=pd.DataFrame(R["Rate"].value_counts()).sort_index(ascending=False)  
y

,Rate
1.000000,78
0.888889,2
0.875000,1
0.857143,3
0.833333,1
0.800000,5
0.777778,3
0.769231,2
0.761905,1
0.750000,17


In [93]:
u=pd.DataFrame(S["Rate"].value_counts()).sort_index(ascending=False)  
u

,Rate
0.727273,1
0.722222,1
0.700000,1
0.692308,1
0.666667,22
0.652174,1


In [94]:
pd.concat([t,y,u], axis=1)    # total, train, test

,Rate,Rate,Rate
1.000000,78,78.0,NaN
0.888889,2,2.0,NaN
0.875000,1,1.0,NaN
0.857143,3,3.0,NaN
0.833333,1,1.0,NaN
0.800000,5,5.0,NaN
0.777778,3,3.0,NaN
0.769231,2,2.0,NaN
0.761905,1,1.0,NaN
0.750000,17,17.0,NaN


In [95]:
init1=0
for r in RL:
    W=gf[gf['Post']==r][['Label']]
    init1 += list(W['Label']).count('1')
init1

520

In [96]:
init2=0
for r in RL:
    W=gf[gf['Post']==r][['Label']]
    init2 += list(W['Label']).count('0')
init2

362

In [97]:
init3=0
for s in SL:
    W=gf[gf['Post']==s][['Label']]
    init3 += list(W['Label']).count('1')
init3

129

In [98]:
init4=0
for s in SL:
    W=gf[gf['Post']==s][['Label']]
    init4 += list(W['Label']).count('0')
init4

86

# Split 6

In [100]:
# Original H, before sorting
H

,Post,Rate,Count
0,1,1.000000,1
1,2,0.000000,0
2,3,1.000000,3
3,4,1.000000,1
4,5,0.800000,15
...,...,...,...
295,296,0.000000,0
296,297,0.000000,0
297,298,0.666667,6
298,299,0.500000,2


In [101]:
H=H.sort_values(by='Rate', ascending=False)
H

,Post,Rate,Count
0,1,1.0,1
136,137,1.0,5
133,134,1.0,3
250,251,1.0,3
129,130,1.0,2
...,...,...,...
154,155,0.0,1
156,157,0.0,2
236,237,0.0,0
235,236,0.0,1


In [102]:
h=H[["Rate","Count"]]
numlist = list(range(len(h)))
h = h.set_index(pd.Index(numlist))
h

,Rate,Count
0,1.0,1
1,1.0,5
2,1.0,3
3,1.0,3
4,1.0,2
...,...,...
295,0.0,1
296,0.0,2
297,0.0,0
298,0.0,1


In [103]:
# pd.set_option('display.max_rows', None)
Accu=[]
for i in range(len(h)):
    if i==0:  Accu.append(h["Count"][0])
    else:  Accu.append(Accu[-1]+h["Count"][i])
# print(len(Accu),Accu)
h["Accu"]=Accu
h

,Rate,Count,Accu
0,1.0,1,1
1,1.0,5,6
2,1.0,3,9
3,1.0,3,12
4,1.0,2,14
...,...,...,...
295,0.0,1,1094
296,0.0,2,1096
297,0.0,0,1096
298,0.0,1,1097


In [104]:
h["%"]=h["Accu"]/1097*100
h

,Rate,Count,Accu,%
0,1.0,1,1,0.091158
1,1.0,5,6,0.546946
2,1.0,3,9,0.820419
3,1.0,3,12,1.093892
4,1.0,2,14,1.276208
...,...,...,...,...
295,0.0,1,1094,99.726527
296,0.0,2,1096,99.908842
297,0.0,0,1096,99.908842
298,0.0,1,1097,100.000000


In [131]:
h["%"][129]

49.86326344576117

In [132]:
# Split 2 equal groups // 0~130 thread take 50.13%
H1=H[:131] # 1st group
H2=H[131:] # 2nd group
H2

,Post,Rate,Count
275,276,0.666667,9
24,25,0.666667,24
139,140,0.666667,3
112,113,0.666667,3
113,114,0.666667,6
...,...,...,...
154,155,0.000000,1
156,157,0.000000,2
236,237,0.000000,0
235,236,0.000000,1


In [133]:
# for H1
for i in range(1,len(H1)+1):
    U1=sum(list(H1['Count'][:i]))
    print("percentage of train data: ", U1/1097)
    # train
    R1=H1[:i]
    # test
    S1=H1[i:]
    RL1=list(R1['Post'].sort_values())
    print(len(RL1))
    SL1=list(S1['Post'].sort_values())
    print(len(SL1))
    init11=0
    for r in RL1:
        W=gf[gf['Post']==r][['Label']]
        init11 += list(W['Label']).count('1')
    print(init11)
    init12=0
    for r in RL1:
        W=gf[gf['Post']==r][['Label']]
        init12 += list(W['Label']).count('0')
    print(init12)
    init13=0
    for s in SL1:
        W=gf[gf['Post']==s][['Label']]
        init13 += list(W['Label']).count('1')
    print(init13)
    init14=0
    for s in SL1:
        W=gf[gf['Post']==s][['Label']]
        init14 += list(W['Label']).count('0')
    print(init14)
    if (init11+init12) == 0:
        print(0)
    else:    print(init11/(init11+init12)) # rate of '1' in train data
    if (init13+init14) == 0:
        print(0)
    else:    print(init13/(init13+init14)) # rate of '1' in test data
    print('*'*100)

percentage of train data:  0.0009115770282588879
1
130
0
1
316
233
0.0
0.575591985428051
****************************************************************************************************
percentage of train data:  0.0054694621695533276
2
129
3
3
313
231
0.5
0.5753676470588235
****************************************************************************************************
percentage of train data:  0.008204193254329991
3
128
5
4
311
230
0.5555555555555556
0.5748613678373382
****************************************************************************************************
percentage of train data:  0.010938924339106655
4
127
8
4
308
230
0.6666666666666666
0.5724907063197026
****************************************************************************************************
percentage of train data:  0.01276207839562443
5
126
10
4
306
230
0.7142857142857143
0.5708955223880597
****************************************************************************************************
perc

272
205
0.6027397260273972
0.570230607966457
****************************************************************************************************
percentage of train data:  0.06836827711941659
42
89
45
30
271
204
0.6
0.5705263157894737
****************************************************************************************************
percentage of train data:  0.06927985414767548
43
88
45
31
271
203
0.5921052631578947
0.5717299578059072
****************************************************************************************************
percentage of train data:  0.07201458523245215
44
87
47
32
269
202
0.5949367088607594
0.5711252653927813
****************************************************************************************************
percentage of train data:  0.07292616226071102
45
86
48
32
268
202
0.6
0.5702127659574469
****************************************************************************************************
percentage of train data:  0.07383773928896992
46
85
48
33
2

76
197
158
0.6102564102564103
0.5549295774647888
****************************************************************************************************
percentage of train data:  0.18413855970829535
83
48
123
79
193
155
0.6089108910891089
0.5545977011494253
****************************************************************************************************
percentage of train data:  0.19051959890610756
84
47
127
82
189
152
0.6076555023923444
0.5542521994134897
****************************************************************************************************
percentage of train data:  0.1959890610756609
85
46
130
85
186
149
0.6046511627906976
0.5552238805970149
****************************************************************************************************
percentage of train data:  0.20054694621695535
86
45
134
86
182
148
0.6090909090909091
0.5515151515151515
****************************************************************************************************
percentage of train dat

298
219
18
15
0.5764023210831721
0.5454545454545454
****************************************************************************************************
percentage of train data:  0.4740200546946217
124
7
300
220
16
14
0.5769230769230769
0.5333333333333333
****************************************************************************************************
percentage of train data:  0.47675478577939834
125
6
301
222
15
12
0.5755258126195029
0.5555555555555556
****************************************************************************************************
percentage of train data:  0.479489516864175
126
5
303
223
13
11
0.5760456273764258
0.5416666666666666
****************************************************************************************************
percentage of train data:  0.49042844120328166
127
4
308
230
8
4
0.5724907063197026
0.6666666666666666
****************************************************************************************************
percentage of train data:  0

In [113]:
# for H2
for i in range(1,len(H2)+1):
    U2=sum(list(H2['Count'][:i]))
    print("percentage of train data: ", U2/1097)
    # train
    R2=H2[:i]
    # test
    S2=H2[i:]
    RL2=list(R2['Post'].sort_values())
    print(len(RL2))
    SL2=list(S2['Post'].sort_values())
    print(len(SL2))
    init21=0
    for r in RL2:
        W=gf[gf['Post']==r][['Label']]
        init21 += list(W['Label']).count('1')
    print(init21)
    init22=0
    for r in RL2:
        W=gf[gf['Post']==r][['Label']]
        init22 += list(W['Label']).count('0')
    print(init22)
    init23=0
    for s in SL2:
        W=gf[gf['Post']==s][['Label']]
        init23 += list(W['Label']).count('1')
    print(init23)
    init24=0
    for s in SL2:
        W=gf[gf['Post']==s][['Label']]
        init24 += list(W['Label']).count('0')
    print(init24)
    if (init1+init2) == 0:
        print(0)
    else:    print(init21/(init21+init22)) # rate of '1' in train data
    if (init23+init24) == 0:
        print(0)
    else:    print(init23/(init23+init24)) # rate of '1' in test data
    print('*'*100)

percentage of train data:  0.008204193254329991
1
168
4
5
329
209
0.4444444444444444
0.6115241635687733
****************************************************************************************************
percentage of train data:  0.0300820419325433
2
167
19
14
314
200
0.5757575757575758
0.6108949416342413
****************************************************************************************************
percentage of train data:  0.032816773017319965
3
166
22
14
311
200
0.6111111111111112
0.6086105675146771
****************************************************************************************************
percentage of train data:  0.03555150410209663
4
165
23
16
310
198
0.5897435897435898
0.610236220472441
****************************************************************************************************
percentage of train data:  0.04102096627164995
5
164
24
21
309
193
0.5333333333333333
0.6155378486055777
*************************************************************************

104
0.6507936507936508
0.5517241379310345
****************************************************************************************************
percentage of train data:  0.292616226071103
42
127
209
112
124
102
0.6510903426791277
0.5486725663716814
****************************************************************************************************
percentage of train data:  0.2944393801276208
43
126
209
114
124
100
0.6470588235294118
0.5535714285714286
****************************************************************************************************
percentage of train data:  0.29626253418413856
44
125
209
116
124
98
0.6430769230769231
0.5585585585585585
****************************************************************************************************
percentage of train data:  0.29808568824065634
45
124
210
117
123
97
0.6422018348623854
0.5590909090909091
****************************************************************************************************
percentage of train data: 

199
34
15
0.6004016064257028
0.6938775510204082
****************************************************************************************************
percentage of train data:  0.4667274384685506
83
86
311
201
22
13
0.607421875
0.6285714285714286
****************************************************************************************************
percentage of train data:  0.47037374658158615
84
85
315
201
18
13
0.6104651162790697
0.5806451612903226
****************************************************************************************************
percentage of train data:  0.4740200546946217
85
84
317
203
16
11
0.6096153846153847
0.5925925925925926
****************************************************************************************************
percentage of train data:  0.47766636280765723
86
83
320
204
13
10
0.6106870229007634
0.5652173913043478
****************************************************************************************************
percentage of train data:  0.4822242

212
9
2
0.6044776119402985
0.8181818181818182
****************************************************************************************************
percentage of train data:  0.4886052871467639
124
45
324
212
9
2
0.6044776119402985
0.8181818181818182
****************************************************************************************************
percentage of train data:  0.4886052871467639
125
44
324
212
9
2
0.6044776119402985
0.8181818181818182
****************************************************************************************************
percentage of train data:  0.4886052871467639
126
43
324
212
9
2
0.6044776119402985
0.8181818181818182
****************************************************************************************************
percentage of train data:  0.48951686417502277
127
42
325
212
8
2
0.6052141527001862
0.8
****************************************************************************************************
percentage of train data:  0.48951686417502277
128


166
3
332
214
1
0
0.608058608058608
1.0
****************************************************************************************************
percentage of train data:  0.4977210574293528
167
2
332
214
1
0
0.608058608058608
1.0
****************************************************************************************************
percentage of train data:  0.4986326344576117
168
1
333
214
0
0
0.6087751371115173
0
****************************************************************************************************
percentage of train data:  0.4986326344576117
169
0
333
214
0
0
0.6087751371115173
0
****************************************************************************************************


In [114]:
v1=116
v2=73

In [115]:
# train
R1=H1[:v1]
R2=H2[:v2]
R=pd.concat([R1,R2])
# test
S1=H1[v1:]
S2=H2[v2:]
S=pd.concat([S1,S2])

In [116]:
R1

,Post,Rate,Count
0,1,1.000000,1
136,137,1.000000,5
133,134,1.000000,3
250,251,1.000000,3
129,130,1.000000,2
...,...,...,...
95,96,0.750000,4
83,84,0.750000,4
99,100,0.727273,11
18,19,0.727273,11


In [117]:
U1=sum(list(R1['Count']))
U2=sum(list(R2['Count']))
U3=sum(list(S1['Count']))
U4=sum(list(S2['Count']))
print(U1, U2, U3, U4)

433 436 117 111


In [118]:
# train thread#
RL=list(R['Post'].sort_values())
print(len(RL),RL)

189 [1, 3, 4, 5, 7, 9, 10, 11, 12, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 37, 38, 39, 41, 43, 44, 45, 46, 47, 51, 52, 53, 55, 56, 58, 60, 63, 64, 66, 68, 69, 73, 74, 77, 79, 81, 82, 84, 85, 88, 89, 90, 91, 92, 94, 96, 97, 98, 100, 101, 104, 105, 106, 108, 109, 111, 112, 113, 114, 115, 116, 117, 119, 120, 121, 122, 125, 126, 127, 129, 130, 132, 133, 134, 136, 137, 138, 139, 140, 143, 148, 149, 152, 153, 156, 160, 161, 163, 164, 166, 167, 169, 172, 173, 177, 178, 179, 182, 183, 186, 188, 189, 190, 191, 193, 194, 195, 196, 197, 198, 200, 203, 204, 206, 210, 212, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 230, 231, 232, 234, 238, 240, 243, 244, 245, 248, 249, 250, 251, 254, 255, 257, 258, 260, 261, 262, 263, 264, 266, 268, 271, 273, 275, 276, 277, 278, 279, 281, 282, 283, 287, 288, 289, 290, 293, 298, 299]


In [119]:
# test thread#
SL=list(S['Post'].sort_values())
print(len(SL),SL)

111 [2, 6, 8, 13, 35, 36, 40, 42, 48, 49, 50, 54, 57, 59, 61, 62, 65, 67, 70, 71, 72, 75, 76, 78, 80, 83, 86, 87, 93, 95, 99, 102, 103, 107, 110, 118, 123, 124, 128, 131, 135, 141, 142, 144, 145, 146, 147, 150, 151, 154, 155, 157, 158, 159, 162, 165, 168, 170, 171, 174, 175, 176, 180, 181, 184, 185, 187, 192, 199, 201, 202, 205, 207, 208, 209, 211, 213, 214, 215, 216, 229, 233, 235, 236, 237, 239, 241, 242, 246, 247, 252, 253, 256, 259, 265, 267, 269, 270, 272, 274, 280, 284, 285, 286, 291, 292, 294, 295, 296, 297, 300]


In [121]:
t=pd.DataFrame(H["Rate"].value_counts()).sort_index(ascending=False) 
t

,Rate
1.000000,78
0.888889,2
0.875000,1
0.857143,3
0.833333,1
0.800000,5
0.777778,3
0.769231,2
0.761905,1
0.750000,17


In [122]:
y=pd.DataFrame(R["Rate"].value_counts()).sort_index(ascending=False)  
y

,Rate
1.000000,78
0.888889,2
0.875000,1
0.857143,3
0.833333,1
0.800000,5
0.777778,3
0.769231,2
0.761905,1
0.750000,17


In [123]:
u=pd.DataFrame(S["Rate"].value_counts()).sort_index(ascending=False)  
u

,Rate
0.727273,1
0.722222,1
0.700000,1
0.692308,1
0.666667,11
0.375000,1
0.363636,1
0.333333,6
0.318182,1
0.285714,1


In [124]:
pd.concat([t,y,u], axis=1)  

,Rate,Rate,Rate
1.000000,78,78.0,NaN
0.888889,2,2.0,NaN
0.875000,1,1.0,NaN
0.857143,3,3.0,NaN
0.833333,1,1.0,NaN
0.800000,5,5.0,NaN
0.777778,3,3.0,NaN
0.769231,2,2.0,NaN
0.761905,1,1.0,NaN
0.750000,17,17.0,NaN


In [125]:
init1=0
for r in RL:
    W=gf[gf['Post']==r][['Label']]
    init1 += list(W['Label']).count('1')
init1

515

In [126]:
init2=0
for r in RL:
    W=gf[gf['Post']==r][['Label']]
    init2 += list(W['Label']).count('0')
init2

354

In [127]:
init3=0
for s in SL:
    W=gf[gf['Post']==s][['Label']]
    init3 += list(W['Label']).count('1')
init3

134

In [128]:
init4=0
for s in SL:
    W=gf[gf['Post']==s][['Label']]
    init4 += list(W['Label']).count('0')
init4

94